# Support Vector Machine (SVM) pour Classification Binaire - Diagnostic du Cancer

Ce notebook implémente les modèles **L1-SVM** et **L2-SVM** pour le diagnostic du cancer (Bénin vs Malin).

## Équations du SVM

### L1-SVM (Primal Form):
$$\min_{w,b} \frac{1}{p}w^T w + C \sum_{i=1}^{p} \max(0, 1 - y_i'(w \cdot x_i + b))$$

### L2-SVM:
$$\min_{w,b} \frac{1}{p}||w||_2^2 + C \sum_{i=1}^{p} \max(0, 1 - y_i'(w \cdot x_i + b))^2$$

Où:
- $w$ : vecteur de poids
- $b$ : biais
- $C$ : paramètre de régularisation
- $y_i' \in \{-1, +1\}$ : labels

**Auteur**: maramchebbi  
**Date**: 2025-11-19

## 1. Installation et Importation

In [ ]:
!pip install -q scikit-learn numpy pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import LinearSVC, SVC
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Bibliothèques importées!!!! avec succès")

## 2. Chargement des Données

In [ ]:
from google.colab import files
print("📁 Veuillez uploader votre fichier data.csv")
uploaded = files.upload()

In [ ]:
# Charger les données
data = pd.read_csv('data.csv')

print("📊 Dimensions du dataset:", data.shape)
print("\n📋 Premières lignes:")
print(data.head())

print("\n📈 Distribution des diagnostics:")
print(data['diagnosis'].value_counts())
print("\n📊 Pourcentage:")
print(data['diagnosis'].value_counts(normalize=True) * 100)

## 3. Préparation des Données

In [ ]:
# Nettoyage et préparation
# 1. Supprimer les colonnes inutiles
if 'id' in data.columns:
    data = data.drop('id', axis=1)

# Supprimer les colonnes vides (Unnamed)
unnamed_cols = [col for col in data.columns if 'Unnamed' in col]
if unnamed_cols:
    print(f"⚠️ Suppression des colonnes vides: {unnamed_cols}")
    data = data.drop(columns=unnamed_cols)

# 2. Encoder le diagnostic (M=1, B=0)
le = LabelEncoder()
data['diagnosis'] = le.fit_transform(data['diagnosis'])
print(f"\n✅ Encodage: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# 3. Séparer features et cible
X = data.drop('diagnosis', axis=1).values
y = data['diagnosis'].values

# Convertir y en {-1, +1} pour le SVM
y_svm = np.where(y == 0, -1, 1)

print(f"\n📊 Dimensions:")
print(f"  X: {X.shape}")
print(f"  y: {y.shape}")
print(f"  Classes SVM: {np.unique(y_svm)}")

# 4. Normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\n✅ Données normalisées")
print(f"  Mean: {X_scaled.mean():.4f}")
print(f"  Std: {X_scaled.std():.4f}")

In [ ]:
# 5. Division train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_svm, test_size=0.2, random_state=42, stratify=y_svm
)

print(f"\n📊 Taille des ensembles:")
print(f"  Train: {X_train.shape[0]} samples ({np.sum(y_train==-1)} Bénin, {np.sum(y_train==1)} Malin)")
print(f"  Test: {X_test.shape[0]} samples ({np.sum(y_test==-1)} Bénin, {np.sum(y_test==1)} Malin)")

## 4. Hyperparamètres selon le Tableau 1

In [ ]:
# Hyperparamètres pour SVM
HYPERPARAMETERS = {
    'batch_size': 128,
    'epochs': 3000,
    'learning_rate': 1e-3,
    'norm': 'L2',
    'svm_c': 5,  # Paramètre C principal
    'max_iter': 5000
}

print("="*60)
print("HYPERPARAMÈTRES DES MODÈLES SVM")
print("="*60)
for param, value in HYPERPARAMETERS.items():
    print(f"{param:.<30} {value}")
print("="*60)

## 5. Visualisation des Hyperparamètres

In [ ]:
# Visualisation
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Hyperparamètres des Modèles SVM', fontsize=16, fontweight='bold')

# SVM C
axes[0].bar(['SVM C'], [HYPERPARAMETERS['svm_c']], color='#2E86AB')
axes[0].set_ylabel('Valeur')
axes[0].set_title('Paramètre C\n(Régularisation)')
axes[0].text(0, HYPERPARAMETERS['svm_c']/2, str(HYPERPARAMETERS['svm_c']), 
             ha='center', va='center', fontsize=14, color='white', fontweight='bold')

# Max Iterations
axes[1].bar(['Max Iter'], [HYPERPARAMETERS['max_iter']], color='#F18F01')
axes[1].set_ylabel('Nombre')
axes[1].set_title('Itérations Maximales')
axes[1].text(0, HYPERPARAMETERS['max_iter']/2, str(HYPERPARAMETERS['max_iter']), 
             ha='center', va='center', fontsize=14, color='white', fontweight='bold')

# Norm Type
norm_map = {'L1': 1, 'L2': 2}
axes[2].bar(['Norm'], [norm_map[HYPERPARAMETERS['norm']]], color='#6A994E')
axes[2].set_ylabel('Type')
axes[2].set_title('Type de Norme')
axes[2].set_ylim([0, 3])
axes[2].text(0, 1, HYPERPARAMETERS['norm'], 
             ha='center', va='center', fontsize=14, color='white', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Implémentation des Modèles SVM

In [ ]:
class SVMModels:
    """
    Classe pour entraîner et évaluer les modèles SVM.
    
    Implémente:
    - L1-SVM: avec norme L1 (penalty='l1')
    - L2-SVM: avec norme L2 (penalty='l2')
    """
    
    def __init__(self, C=5, max_iter=5000):
        self.C = C
        self.max_iter = max_iter
        self.models = {}
        self.results = {}
    
    def train_l1_svm(self, X_train, y_train):
        """
        Entraîne le L1-SVM.
        
        Équation (19):
        min (1/p)w^T w + C * sum(max(0, 1 - y_i'(wx_i + b)))
        """
        print("\n🔄 Entraînement du L1-SVM...")
        print(f"  Paramètres: C={self.C}, penalty='l1', loss='squared_hinge'")
        
        # L1-SVM avec dual=False (forme primale)
        self.models['L1-SVM'] = LinearSVC(
            C=self.C,
            penalty='l1',
            loss='squared_hinge',
            dual=False,  # Nécessaire pour penalty='l1'
            max_iter=self.max_iter,
            random_state=42,
            verbose=1
        )
        
        self.models['L1-SVM'].fit(X_train, y_train)
        print("✅ L1-SVM entraîné avec succès!")
        
        # Nombre de features sélectionnées (sparsité du L1)
        n_features_selected = np.sum(np.abs(self.models['L1-SVM'].coef_) > 1e-5)
        print(f"📊 Features sélectionnées (sparsité L1): {n_features_selected}/{X_train.shape[1]}")
    
    def train_l2_svm(self, X_train, y_train):
        """
        Entraîne le L2-SVM.
        
        Équation (20):
        min (1/p)||w||_2^2 + C * sum(max(0, 1 - y_i'(wx_i + b))^2)
        """
        print("\n🔄 Entraînement du L2-SVM...")
        print(f"  Paramètres: C={self.C}, penalty='l2', loss='squared_hinge'")
        
        # L2-SVM (plus standard et différentiable)
        self.models['L2-SVM'] = LinearSVC(
            C=self.C,
            penalty='l2',
            loss='squared_hinge',
            dual=True,
            max_iter=self.max_iter,
            random_state=42,
            verbose=1
        )
        
        self.models['L2-SVM'].fit(X_train, y_train)
        print("✅ L2-SVM entraîné avec succès!")
    
    def evaluate(self, X_test, y_test, model_name):
        """
        Évalue un modèle SVM.
        """
        model = self.models[model_name]
        y_pred = model.predict(X_test)
        
        # Métriques
        accuracy = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        tn, fp, fn, tp = conf_matrix.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
        
        self.results[model_name] = {
            'accuracy': accuracy,
            'confusion_matrix': conf_matrix,
            'tpr': tpr,
            'tnr': tnr,
            'fpr': fpr,
            'fnr': fnr,
            'predictions': y_pred
        }
        
        return self.results[model_name]
    
    def print_results(self, model_name):
        """Affiche les résultats d'un modèle."""
        results = self.results[model_name]
        
        print(f"\n{'='*60}")
        print(f"RÉSULTATS: {model_name}")
        print(f"{'='*60}")
        print(f"\n✅ Accuracy: {results['accuracy']*100:.2f}%")
        print(f"\n📊 Métriques détaillées:")
        print(f"  TPR (Sensibilité):     {results['tpr']*100:.2f}%")
        print(f"  TNR (Spécificité):     {results['tnr']*100:.2f}%")
        print(f"  FPR (Faux Positifs):   {results['fpr']*100:.2f}%")
        print(f"  FNR (Faux Négatifs):   {results['fnr']*100:.2f}%")
        print(f"{'='*60}")
    
    def get_hyperplane_equation(self, model_name):
        """Retourne l'équation de l'hyperplan optimal."""
        model = self.models[model_name]
        w = model.coef_[0]
        b = model.intercept_[0]
        
        print(f"\n📐 Équation de l'hyperplan {model_name}:")
        print(f"  f(w,x) = w · x + b = 0")
        print(f"  Dimensions de w: {w.shape}")
        print(f"  Biais b: {b:.4f}")
        print(f"  Norme ||w||: {np.linalg.norm(w):.4f}")
        
        return w, b

## 7. Entraînement des Modèles

In [ ]:
# Créer l'instance
svm_models = SVMModels(C=HYPERPARAMETERS['svm_c'], max_iter=HYPERPARAMETERS['max_iter'])

print("="*60)
print("ENTRAÎNEMENT DES MODÈLES SVM")
print("="*60)

In [ ]:
# Entraîner L1-SVM
svm_models.train_l1_svm(X_train, y_train)

In [ ]:
# Entraîner L2-SVM
svm_models.train_l2_svm(X_train, y_train)

## 8. Évaluation des Modèles

In [ ]:
# Évaluer L1-SVM
results_l1 = svm_models.evaluate(X_test, y_test, 'L1-SVM')
svm_models.print_results('L1-SVM')
w_l1, b_l1 = svm_models.get_hyperplane_equation('L1-SVM')

In [ ]:
# Évaluer L2-SVM
results_l2 = svm_models.evaluate(X_test, y_test, 'L2-SVM')
svm_models.print_results('L2-SVM')
w_l2, b_l2 = svm_models.get_hyperplane_equation('L2-SVM')

## 9. Visualisations

In [ ]:
# Matrices de confusion
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Matrices de Confusion - Modèles SVM', fontsize=16, fontweight='bold')

# L1-SVM
sns.heatmap(results_l1['confusion_matrix'], annot=True, fmt='d', cmap='Blues',
            xticklabels=['Bénin (-1)', 'Malin (+1)'],
            yticklabels=['Bénin (-1)', 'Malin (+1)'],
            ax=axes[0], annot_kws={"size": 14})
axes[0].set_xlabel('Classe Prédite', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Classe Réelle', fontsize=12, fontweight='bold')
axes[0].set_title(f'L1-SVM\nAccuracy: {results_l1["accuracy"]*100:.2f}%', fontsize=14)

# L2-SVM
sns.heatmap(results_l2['confusion_matrix'], annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Bénin (-1)', 'Malin (+1)'],
            yticklabels=['Bénin (-1)', 'Malin (+1)'],
            ax=axes[1], annot_kws={"size": 14})
axes[1].set_xlabel('Classe Prédite', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Classe Réelle', fontsize=12, fontweight='bold')
axes[1].set_title(f'L2-SVM\nAccuracy: {results_l2["accuracy"]*100:.2f}%', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# Comparaison des métriques
metrics = ['Accuracy', 'TPR', 'TNR', 'FPR', 'FNR']
l1_values = [results_l1['accuracy']*100, results_l1['tpr']*100, 
             results_l1['tnr']*100, results_l1['fpr']*100, results_l1['fnr']*100]
l2_values = [results_l2['accuracy']*100, results_l2['tpr']*100, 
             results_l2['tnr']*100, results_l2['fpr']*100, results_l2['fnr']*100]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(14, 7))
bars1 = ax.bar(x - width/2, l1_values, width, label='L1-SVM', color='#2E86AB')
bars2 = ax.bar(x + width/2, l2_values, width, label='L2-SVM', color='#F18F01')

ax.set_ylabel('Pourcentage (%)', fontsize=14, fontweight='bold')
ax.set_title('Comparaison des Performances: L1-SVM vs L2-SVM', fontsize=16, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics, fontsize=12)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

# Ajouter les valeurs
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## 10. Comparaison avec le Tableau 2

In [ ]:
# Résultats attendus du Tableau 2
comparison_data = {
    'Model': ['SVM (Article)', 'L1-SVM (Notre)', 'L2-SVM (Notre)'],
    'Accuracy': [96.09, results_l1['accuracy']*100, results_l2['accuracy']*100],
    'TPR': [97.53, results_l1['tpr']*100, results_l2['tpr']*100],
    'TNR': [93.62, results_l1['tnr']*100, results_l2['tnr']*100],
    'FPR': [6.38, results_l1['fpr']*100, results_l2['fpr']*100],
    'FNR': [2.47, results_l1['fnr']*100, results_l2['fnr']*100]
}

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*70)
print("COMPARAISON AVEC LE TABLEAU 2")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

## 11. Optimisation des Hyperparamètres

In [ ]:
# Grid Search pour trouver le meilleur C
print("\n🔍 Recherche du meilleur paramètre C...")

param_grid = {
    'C': [0.1, 0.5, 1, 2, 5, 10, 20, 50]
}

# Pour L2-SVM
grid_search_l2 = GridSearchCV(
    LinearSVC(penalty='l2', loss='squared_hinge', dual=True, max_iter=5000, random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search_l2.fit(X_train, y_train)

print(f"\n✅ Meilleur C pour L2-SVM: {grid_search_l2.best_params_['C']}")
print(f"✅ Meilleure accuracy (CV): {grid_search_l2.best_score_*100:.2f}%")

# Résultats de tous les C testés
results_grid = pd.DataFrame(grid_search_l2.cv_results_)
print("\n📊 Résultats pour différentes valeurs de C:")
print(results_grid[['param_C', 'mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False))

In [ ]:
# Visualisation de l'impact de C
plt.figure(figsize=(12, 6))
plt.plot(results_grid['param_C'], results_grid['mean_test_score']*100, 
         marker='o', linewidth=2, markersize=8, color='#2E86AB')
plt.fill_between(results_grid['param_C'], 
                 (results_grid['mean_test_score'] - results_grid['std_test_score'])*100,
                 (results_grid['mean_test_score'] + results_grid['std_test_score'])*100,
                 alpha=0.3, color='#2E86AB')
plt.xlabel('Paramètre C', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy (%)', fontsize=14, fontweight='bold')
plt.title('Impact du Paramètre C sur la Performance du L2-SVM', fontsize=16, fontweight='bold')
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 12. Sauvegarde des Modèles

In [ ]:
import pickle

print("\n💾 Sauvegarde des modèles...")

# Sauvegarder les modèles
with open('l1_svm_model.pkl', 'wb') as f:
    pickle.dump(svm_models.models['L1-SVM'], f)
print("✓ L1-SVM sauvegardé: l1_svm_model.pkl")

with open('l2_svm_model.pkl', 'wb') as f:
    pickle.dump(svm_models.models['L2-SVM'], f)
print("✓ L2-SVM sauvegardé: l2_svm_model.pkl")

# Sauvegarder le scaler
with open('svm_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✓ Scaler sauvegardé: svm_scaler.pkl")

# Sauvegarder les hyperparamètres
with open('svm_hyperparams.pkl', 'wb') as f:
    pickle.dump(HYPERPARAMETERS, f)
print("✓ Hyperparamètres sauvegardés: svm_hyperparams.pkl")

print("\n✅ Tous les modèles ont été sauvegardés!")

In [ ]:
# Télécharger les fichiers
from google.colab import files

print("📥 Téléchargement des fichiers...")
files.download('l1_svm_model.pkl')
files.download('l2_svm_model.pkl')
files.download('svm_scaler.pkl')
files.download('svm_hyperparams.pkl')
print("✅ Téléchargement terminé!")

## 13. Résumé et Conclusion

In [ ]:
print("\n" + "="*70)
print("📊 RÉSUMÉ DES MODÈLES SVM")
print("="*70)

print("\n🎯 Modèles Implémentés:")
print("  1. L1-SVM (Forme Primale - Équation 19)")
print("     - Norme: L1 (Manhattan)")
print("     - Loss: Hinge Standard")
print("     - Sparsité: Oui (sélection de features)")
print(f"     - Accuracy: {results_l1['accuracy']*100:.2f}%")

print("\n  2. L2-SVM (Équation 20)")
print("     - Norme: L2 (Euclidienne)")
print("     - Loss: Squared Hinge")
print("     - Différentiable: Oui")
print(f"     - Accuracy: {results_l2['accuracy']*100:.2f}%")

print("\n📈 Comparaison:")
print(f"  Meilleur modèle: {'L1-SVM' if results_l1['accuracy'] > results_l2['accuracy'] else 'L2-SVM'}")
print(f"  Différence d'accuracy: {abs(results_l1['accuracy'] - results_l2['accuracy'])*100:.2f}%")

print("\n🔑 Points Clés:")
print("  ✓ L1-SVM: Meilleur pour la sélection de features (sparsité)")
print("  ✓ L2-SVM: Plus stable, différentiable, généralement plus performant")
print("  ✓ Hyperplan optimal: f(w,x) = w·x + b = 0")
print(f"  ✓ Paramètre C optimal: {grid_search_l2.best_params_['C']}")

print("\n💾 Fichiers Sauvegardés:")
print("  ✓ l1_svm_model.pkl")
print("  ✓ l2_svm_model.pkl")
print("  ✓ svm_scaler.pkl")
print("  ✓ svm_hyperparams.pkl")

print("\n✅ Modèles prêts pour la production!")
print("="*70)

## 📚 Notes Techniques

### Différences entre L1-SVM et L2-SVM:

| Caractéristique | L1-SVM | L2-SVM |
|----------------|---------|--------|
| **Norme** | Manhattan ($||w||_1$) | Euclidienne ($||w||_2^2$) |
| **Loss Function** | Hinge: $\max(0, 1-y'(wx+b))$ | Squared Hinge: $\max(0, 1-y'(wx+b))^2$ |
| **Différentiabilité** | Non (équation 19) | Oui (équation 20) |
| **Sparsité** | Oui (certains $w_i = 0$) | Non |
| **Sélection features** | Automatique | Non |
| **Stabilité** | Moyenne | Élevée |
| **Usage** | Haute dimension, features redondantes | Général, plus robuste |

### Équations Implémentées:

**L1-SVM (Équation 19):**
$$\min_{w,b} \frac{1}{p}w^T w + C \sum_{i=1}^{p} \max(0, 1 - y_i'(w \cdot x_i + b))$$

**L2-SVM (Équation 20):**
$$\min_{w,b} \frac{1}{p}||w||_2^2 + C \sum_{i=1}^{p} \max(0, 1 - y_i'(w \cdot x_i + b))^2$$

### Hyperplan Optimal:
L'hyperplan séparant les deux classes est défini par:
$$f(w,x) = w \cdot x + b = 0$$

Les prédictions sont obtenues par:
$$y' = \text{sign}(w \cdot x + b)$$

---

**Créé par**: maramchebbi  
**Date**: 2025-11-19  
**Référence**: Section 2.4.6 Support Vector Machine